# process_data.ipynb

## Input a raw data csv, and prepare it for analysis

Cleaning steps:
- Handle a few cases where the first and last rows contain 'bad' values
- Convert hours:minutes time column into a datetime index

Processing steps:
- Resample data from ~1 record / second to 1 record / hour granularity, taking 95th, 50th (median) and 5th quantiles

In [1]:
# Imports:
import pandas as pd

In [2]:
# input and output file names
input_file = "dataset1.csv"
output_file = "dataset1_clean.csv"

In [3]:
# when reading full dataset, skip last 2 rows because they are NaN
raw_data = pd.read_csv(input_file, skipfooter=2)

C:\Users\zpatel\Miniconda3\envs\cs229\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  


In [4]:
def add_datetime_index(dataframe, print_status):
    # convert Minute:Second time stamps into proper datetime typed Time column.
    raw_data.reset_index(inplace=True)
    if print_status:
        print("Reset Index:\n", dataframe.head())
    
    dataframe["Time"] = dataframe["Time"].astype(str).apply(lambda x: x[:-2])
    dataframe["Hour"] = dataframe["index"].floordiv(3600).apply(lambda x: str(x) if x >=10 else "0" + str(x))
    dataframe["Day"] = dataframe["Hour"].astype(int).floordiv(24).apply(lambda x: str(x + 1) if (x + 1) >=10 else "0" + str(x + 1))
    dataframe["Hour"] = dataframe["Hour"].apply(lambda x: int(x) % 24)
    dataframe["Time"] = dataframe["Day"].astype(str) + ":" + dataframe["Hour"].astype(str) + ":" + dataframe["Time"].astype(str)
    dataframe.set_index("index", inplace=True, drop=True)
   
    # %M:%S format turns a reading into minutes:seconds and drops the fractional seconds (since sampling period is 2s)
    dataframe['Time'] = pd.to_datetime(dataframe['Time'], format="%d:%H:%M:%S", exact=False)
    dataframe['Seconds'] = dataframe['Time'].dt.second
    
    if print_status:
        print("Converted Time to DateTime:\n", dataframe.head())

In [5]:
# apply datetime index conversion
add_datetime_index(raw_data, print_status=True)

Reset Index:
    index     Time  DAC S0  SO (volts)  DAC S1  S1 (volts)  DAC S2  S2 (volts)  \
0      0  00:00.9       2         2.0   0.499         0.5     0.5         0.5   
1      1  00:01.9       2         2.0   0.499         0.5     0.5         0.5   
2      2  00:02.9       2         2.0   0.499         0.5     0.5         0.5   
3      3  00:03.9       2         2.0   0.499         0.5     0.5         0.5   
4      4  00:04.9       2         2.0   0.499         0.5     0.5         0.5   

   DAC S3  S3 (volts)  ...  Extractor 3  Extractor 4  Extractor 5  \
0     0.5         0.5  ...      -176.38      -225.05      -192.70   
1     0.5         0.5  ...      -176.35      -225.05      -192.73   
2     0.5         0.5  ...      -176.41      -225.02      -192.70   
3     0.5         0.5  ...      -176.41      -224.99      -192.73   
4     0.5         0.5  ...      -176.35      -225.02      -192.70   

   Extractor 6  BLA 1  BLA 2  BLA 3  BLA 4  BLA 5  BLA 6  
0      -183.08 -30.06 -35

In [6]:
# drop any rows containing NaN values
# since we will later resample by quartile this should be an acceptable compromise
raw_data.dropna(inplace=True)


In [7]:
# Helper functions for resampling
def quant(q):
    # spits out a quantile generating function for using 
    return lambda x: x.quantile(q)

In [8]:
# resample data by hour

# first, drop hour, day and seconds columns
resampled_data = (
    raw_data.drop(columns=["Hour", "Day", "Seconds"])
    .resample(rule="min", on="Time", )
    .agg([quant(.05), 'median', quant(.95)])
    .rename({"<lambda_0>":"5th quantile", "<lambda_1>": "95th quantile"})
)

print("Old #rows: %d, Filtered #rows: %d" %(len(raw_data), len(resampled_data)))

# Check that index was updated properly
resampled_data.head()

Old #rows: 775215, Filtered #rows: 12960


DAC S0                   SO (volts)                    \
                    <lambda_0> median <lambda_1> <lambda_0> median <lambda_1>   
Time                                                                            
1900-01-01 00:00:00        2.0    2.0        2.0        2.0    2.0        2.0   
1900-01-01 00:01:00        2.0    2.0        2.0        2.0    2.0        2.0   
1900-01-01 00:02:00        2.0    2.0        2.0        2.0    2.0        2.0   
1900-01-01 00:03:00        2.0    2.0        2.0        2.0    2.0        2.0   
1900-01-01 00:04:00        2.0    2.0        2.0        2.0    2.0        2.0   

                        DAC S1                   S1 (volts)  ...      BLA 3  \
                    <lambda_0> median <lambda_1> <lambda_0>  ... <lambda_1>   
Time                                                         ...              
1900-01-01 00:00:00      0.499  0.499      0.499        0.5  ...   -35.1800   
1900-01-01 00:01:00      0.499  0.499      0.499        0.5  ...   -36.1895   
1900-01-01 00:02:00      0.499  0.499      0.499        0.5  ...   -36.1270   
1900-01-01 00:03:00      0.499  0.499      0.499        0.5  ...   -36.0400   
1900-01-01 00:04:00      0.499  0.499      0.499        0.5  ...   -35.9800   

                         BLA 4                        BLA 5                    \
                    <lambda_0> median <lambda_1> <lambda_0> median <lambda_1>   
Time                                                                            
1900-01-01 00:00:00   -38.7000 -38.57    -38.022     -35.22 -35.16    -34.889   
1900-01-01 00:01:00   -38.6300 -38.57    -38.510     -35.22 -35.10    -35.030   
1900-01-01 00:02:00   -38.6000 -38.54    -38.480     -35.19 -35.13    -35.030   
1900-01-01 00:03:00   -38.6315 -38.51    -38.450     -35.16 -35.06    -34.970   
1900-01-01 00:04:00   -38.6000 -38.51    -38.480     -35.19 -35.10    -35.030   

                         BLA 6                    
                    <lambda_0> median <lambda_1>  
Time                                              
1900-01-01 00:00:00   -33.3510 -33.20   -33.0800  
1900-01-01 00:01:00   -33.3900 -33.30   -33.1985  
1900-01-01 00:02:00   -33.2600 -33.14   -33.0800  
1900-01-01 00:03:00   -33.3015 -33.17   -33.0800  
1900-01-01 00:04:00   -33.3000 -33.17   -33.1100  

[5 rows x 141 columns]

In [9]:
# flatten multi level columns created by groupby
resampled_data.columns = list(map('_'.join, resampled_data.columns.values))
resampled_data.columns = list(map(lambda x: x.replace("<lambda_0>", "95th_quantile"), resampled_data.columns.values))
resampled_data.columns = list(map(lambda x: x.replace("<lambda_1>", "5th_quantile"), resampled_data.columns.values))
resampled_data.columns = list(map(lambda x: x.replace(" ", "_"), resampled_data.columns.values))

resampled_data.head()

,DAC_S0_95th_quantile,DAC_S0_median,DAC_S0_5th_quantile,SO_(volts)_95th_quantile,SO_(volts)_median,SO_(volts)_5th_quantile,DAC_S1_95th_quantile,DAC_S1_median,DAC_S1_5th_quantile,S1_(volts)_95th_quantile,...,BLA_3_5th_quantile,BLA_4_95th_quantile,BLA_4_median,BLA_4_5th_quantile,BLA_5_95th_quantile,BLA_5_median,BLA_5_5th_quantile,BLA_6_95th_quantile,BLA_6_median,BLA_6_5th_quantile
Time,,,,,,,,,,,,,,,,,,,,,
1900-01-01 00:00:00,2.0,2.0,2.0,2.0,2.0,2.0,0.499,0.499,0.499,0.5,...,-35.1800,-38.7000,-38.57,-38.022,-35.22,-35.16,-34.889,-33.3510,-33.20,-33.0800
1900-01-01 00:01:00,2.0,2.0,2.0,2.0,2.0,2.0,0.499,0.499,0.499,0.5,...,-36.1895,-38.6300,-38.57,-38.510,-35.22,-35.10,-35.030,-33.3900,-33.30,-33.1985
1900-01-01 00:02:00,2.0,2.0,2.0,2.0,2.0,2.0,0.499,0.499,0.499,0.5,...,-36.1270,-38.6000,-38.54,-38.480,-35.19,-35.13,-35.030,-33.2600,-33.14,-33.0800
1900-01-01 00:03:00,2.0,2.0,2.0,2.0,2.0,2.0,0.499,0.499,0.499,0.5,...,-36.0400,-38.6315,-38.51,-38.450,-35.16,-35.06,-34.970,-33.3015,-33.17,-33.0800
1900-01-01 00:04:00,2.0,2.0,2.0,2.0,2.0,2.0,0.499,0.499,0.499,0.5,...,-35.9800,-38.6000,-38.51,-38.480,-35.19,-35.10,-35.030,-33.3000,-33.17,-33.1100


In [10]:
# delete 39 rows that are entirely blank from rows 12900 to 12938
resampled_data.dropna(inplace=True)

In [11]:
# save new dataframe to disk
resampled_data.to_csv(output_file)


